<a href="https://colab.research.google.com/github/pc-vasconcelos/hippo/blob/main/rcdn_eme_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import datetime
from time import time
import google.protobuf.json_format
import logging
import pandas as pd

In [26]:
df_net_raw = pd.read_excel('/content/EME_RC_Files_NetworkFile (10).xlsx', sheet_name="EME Network")
df_net_raw = df_net_raw[(df_net_raw['Region'] == 'EME') & (df_net_raw['InSeason'] == 'Y')]
df_net_raw = df_net_raw[['Specie', 'Flow', 'Sales Country / Country of Destination', 'Production Country', 'Production Location', 'Stage 1: Distribution Lead Time',
                  'Country: Overflow Warehouse 1', 'Overflow Warehouse 1', 'Stage 2: Distribution Lead Time ', 'Country: Overflow Warehouse 2', 'Overflow Warehouse 2',
                  'Stage 3: Distribution Lead Time', 'Pre Positioning DC Country', 'Pre Positioning DC', 'InSeason', 'FutureSeason',
                  'Documentation', 'Customs/Import']]

# Rename columns
rename_columns = ['Specie', 'Flow', 'COD', 'ProductionCountry', 'ProductionLocation', 'TransitLeadTimeDaysLeg1', 'OverflowWhsLeg1Country', 'OverflowWhsLeg1Site',
'TransitLeadTimeDaysLeg2', 'OverflowWhsLeg2Country', 'OverflowWhsLeg2Site', 'TransitLeadTimeDaysLeg3', 'PrepoDCLeg3Country', 'PrepoDCLeg3Site', 'InSeason', 'FutureSeason',
'ExportDocumentation', 'CustomTime']
df_net = df_net_raw
df_net.columns = rename_columns

# Add calculated columns
df_net.insert(5, 'QAStatusLeg1', df_net.apply(
            lambda row: '' if pd.isnull(row['OverflowWhsLeg1Country']) else (
                        'DTOK' if row['ProductionCountry'] == row['OverflowWhsLeg1Country'] else (
                        'XTOK' if pd.notnull(row['OverflowWhsLeg1Country']) else '')), axis=1))

df_net.insert(9, 'QAStatusLeg2', df_net.apply(
            lambda row: '' if (pd.isnull(row['OverflowWhsLeg1Country']) or pd.isnull(row['OverflowWhsLeg2Country'])) else (
                        'DTOK' if (row['QAStatusLeg1'] != 'DTOK') &
                                ((row['ProductionCountry'] == row['OverflowWhsLeg2Country']) or
                                (row['OverflowWhsLeg1Country'] == row['OverflowWhsLeg2Country'])) else (
                        'XTOK' if (row['QAStatusLeg1'] != 'XTOK') else '')), axis=1))

df_net.insert(13, 'QAStatusLeg3', df_net.apply(
            lambda row: 'UnRes' if (row['QAStatusLeg1'] in ['DTOK', 'XTOK']) & (row['QAStatusLeg2'] in ['DTOK', 'XTOK']) else (
                        'UnRes' if (row['OverflowWhsLeg2Country'] == row['PrepoDCLeg3Country']) & ((row['QAStatusLeg1'] == 'DTOK') or (row['QAStatusLeg2'] == 'DTOK')) else (
                        'DTOK' if (row['QAStatusLeg1'] != 'DTOK') & (row['QAStatusLeg2'] != 'DTOK') &
                                ((row['ProductionCountry'] == row['PrepoDCLeg3Country']) or
                                (row['OverflowWhsLeg1Country'] == row['PrepoDCLeg3Country']) or
                                (row['OverflowWhsLeg2Country'] == row['PrepoDCLeg3Country'])) else (
                        'XTOK' if (row['QAStatusLeg1'] != 'XTOK') & (row['QAStatusLeg2'] != 'XTOK') else 'UnRes'))), axis=1))

# Concatenate Site with Country
df_net['OverflowWhsLeg1Site'] = df_net.apply(
    lambda row: pd.NA if pd.isna(row['OverflowWhsLeg1Site']) else
                str(row['OverflowWhsLeg1Site'])[:4] + '_' + str(row['COD']) if row['Flow'] == 'FERT' and pd.notnull(row['COD']) and str(row['COD']) not in ('STL', '', ' ') else
                str(row['OverflowWhsLeg1Site'])[:4] + '_SUP1' if str(row['Flow'])[:4] == 'HALB' else
                str(row['OverflowWhsLeg1Site'])[:4], axis=1)

df_net['OverflowWhsLeg2Site'] = df_net.apply(
    lambda row: pd.NA if pd.isna(row['OverflowWhsLeg2Site']) else
                str(row['OverflowWhsLeg2Site'])[:4] + '_' + str(row['COD']) if row['Flow'] == 'FERT' and pd.notnull(row['COD']) and str(row['COD']) not in ('STL', '', ' ') else
                str(row['OverflowWhsLeg2Site'])[:4] + '_SUP2' if str(row['Flow'])[:4] == 'HALB' else
                str(row['OverflowWhsLeg2Site'])[:4], axis=1)

df_net['PrepoDCLeg3Site'] = df_net.apply(
    lambda row: pd.NA if pd.isna(row['PrepoDCLeg3Site']) else
                str(row['PrepoDCLeg3Site'])[:4] + '_' + str(row['COD']) if row['Flow'] in ('FERT', 'HAWA') and pd.notnull(row['COD']) and str(row['COD']) not in ('STL', '', ' ') else
                str(row['PrepoDCLeg3Site'])[:4], axis=1)

# Add empty columns
df_net.insert(6, 'QALeadTimeDaysLeg1', '')
df_net.insert(11, 'QALeadTimeDaysLeg2', '')
df_net.insert(16, 'QALeadTimeDaysLeg3', '')
df_net.insert(20, 'TotalLeadTimeDaysUnR', '')
df_net.insert(21, 'CustomsLeadTimeDays', 0)
df_net.insert(22, 'CertificationLeadTimeDays', 0)
df_net.insert(23, 'FumigationLeadTimeDays', '')
df_net.insert(24, 'RemainingLeadTimeDaysUnR', '')
df_net.insert(25, 'LeadTimeDaysTotal', 0)

df_net['CustomsLeadTimeDays'] = df_net['CustomTime']
df_net['CertificationLeadTimeDays'] = df_net['ExportDocumentation']

# Read QALeadTime file
df_leadtime = pd.read_excel('/content/EME_RC_Files_EME QA LeadTime (8).xlsx', sheet_name='Sheet1')

# Field conversions
df_net['OverflowWhsLeg2Country'] = df_net['OverflowWhsLeg2Country'].fillna('').astype(pd.StringDtype())
df_leadtime['DestinationCountry'] = df_leadtime['DestinationCountry'].fillna('').astype(pd.StringDtype())
df_net['PrepoDCLeg3Plant'] = df_net['PrepoDCLeg3Site'].astype(str).str[:4]

# Replace values in Stage
df_leadtime['Stage'] = df_leadtime['Stage'].replace({'CS-KGWB': 'HALB_CS', 'DS-KGWB': 'HALB_DS'})

# Merge Network and QALeadTime
merged_df_0 = pd.merge(
    df_net,
    df_leadtime,
    how='left',
    left_on=['Specie', 'ProductionCountry', 'ProductionLocation', 'COD', 'Flow'],
    right_on=['SPECIE', 'ProcessingCountry', 'plant', 'DestinationCountry', 'Stage']
)

merged_df_1 = pd.merge(
    merged_df_0,
    df_leadtime,
    how='left',
    left_on=['Specie', 'ProductionCountry', 'ProductionLocation', 'OverflowWhsLeg1Country', 'Flow'],
    right_on=['SPECIE', 'ProcessingCountry', 'plant', 'DestinationCountry', 'Stage'],
    suffixes=('', '_Whs1')
)

merged_df_2 = pd.merge(
    merged_df_1,
    df_leadtime,
    how='left',
    left_on=['Specie', 'ProductionCountry', 'ProductionLocation', 'OverflowWhsLeg2Country', 'Flow'],
    right_on=['SPECIE', 'ProcessingCountry', 'plant', 'DestinationCountry', 'Stage'],
    suffixes=('', '_Whs2')
)

merged_df_3 = pd.merge(
    merged_df_2,
    df_leadtime,
    how='left',
    left_on=['Specie', 'ProductionCountry', 'ProductionLocation', 'PrepoDCLeg3Country', 'Flow'],
    right_on=['SPECIE', 'ProcessingCountry', 'plant', 'DestinationCountry', 'Stage'],
    suffixes=('', '_Prepo3')
)

# merge for when ProductionLocation is empty and flow is FERT
df_leadtime['plant'] = df_leadtime['plant'].fillna('').astype(pd.StringDtype())

merged_df = pd.merge(
    merged_df_3,
    df_leadtime,
    how='left',
    left_on=['Specie', 'ProductionCountry', 'PrepoDCLeg3Plant', 'PrepoDCLeg3Country', 'Flow'],
    right_on=['SPECIE', 'ProcessingCountry', 'plant', 'DestinationCountry', 'Stage'],
    suffixes=('', '_Prepo3Site')
)

# Fill empty values with 0
def df_fill_zero(df, columns):
    for column in columns:
        df[column] = df[column].fillna(0).astype(int)
    return df

columns_fill_zero = ['DTOK Time', 'XTOK Time', 'Total Lead Time', 'TransitLeadTimeDaysLeg1', 'TransitLeadTimeDaysLeg2', 'TransitLeadTimeDaysLeg3',
                    'ExportDocumentation', 'CustomTime', 'DTOK Time_Whs1', 'XTOK Time_Whs1', 'DTOK Time_Whs2', 'XTOK Time_Whs2', 'DTOK Time_Prepo3', 'XTOK Time_Prepo3']

merged_df = df_fill_zero(merged_df, columns_fill_zero)

# Calculate LeadTimes after merge
merged_df['QALeadTimeDaysLeg1'] = merged_df.apply(
    lambda row: row['DTOK Time'] if (row['QAStatusLeg1'] == 'DTOK') & (row['Flow'] == 'FERT') else
                row['XTOK Time'] if (row['QAStatusLeg1'] == 'XTOK') & (row['Flow'] == 'FERT') else
                row['DTOK Time_Whs1'] if (row['QAStatusLeg1'] == 'DTOK') & (str(row['Flow'])[:4] == 'HALB') else
                row['XTOK Time_Whs1'] if (row['QAStatusLeg1'] == 'XTOK') & (str(row['Flow'])[:4] == 'HALB') else
                0, axis=1)

merged_df['QALeadTimeDaysLeg2'] = merged_df.apply(
    lambda row: (row['DTOK Time'] - row['QALeadTimeDaysLeg1'] - row['TransitLeadTimeDaysLeg1']) if (row['QAStatusLeg2'] == 'DTOK') & (row['Flow'] == 'FERT') else
                (row['XTOK Time'] - row['QALeadTimeDaysLeg1'] - row['TransitLeadTimeDaysLeg1']) if (row['QAStatusLeg2'] == 'XTOK') & (row['Flow'] == 'FERT') else
                (row['DTOK Time_Whs2'] - row['QALeadTimeDaysLeg1'] - row['TransitLeadTimeDaysLeg1']) if (row['QAStatusLeg2'] == 'DTOK') & (str(row['Flow'])[:4] == 'HALB') else
                (row['XTOK Time_Whs2'] - row['QALeadTimeDaysLeg1'] - row['TransitLeadTimeDaysLeg1']) if (row['QAStatusLeg2'] == 'XTOK') & (str(row['Flow'])[:4] == 'HALB') else
                0, axis=1)
merged_df['QALeadTimeDaysLeg2'] = merged_df.apply(lambda row: 0 if row['QALeadTimeDaysLeg2'] < 0 else row['QALeadTimeDaysLeg2'], axis=1)

merged_df['QALeadTimeDaysLeg3'] = merged_df.apply(
    lambda row: (row['DTOK Time'] - row['QALeadTimeDaysLeg1'] - row['TransitLeadTimeDaysLeg1'] - row['QALeadTimeDaysLeg2'] - row['TransitLeadTimeDaysLeg2']) if (row['QAStatusLeg3'] == 'DTOK') & (row['Flow'] == 'FERT') else
                (row['XTOK Time'] - row['QALeadTimeDaysLeg1'] - row['TransitLeadTimeDaysLeg1'] - row['QALeadTimeDaysLeg2'] - row['TransitLeadTimeDaysLeg2']) if (row['QAStatusLeg3'] == 'XTOK') & (row['Flow'] == 'FERT') else
                (row['DTOK Time_Prepo3'] - row['QALeadTimeDaysLeg1'] - row['TransitLeadTimeDaysLeg1'] - row['QALeadTimeDaysLeg2'] - row['TransitLeadTimeDaysLeg2']) if (row['QAStatusLeg3'] == 'DTOK') & (str(row['Flow'])[:4] == 'HALB') else
                (row['XTOK Time_Prepo3'] - row['QALeadTimeDaysLeg1'] - row['TransitLeadTimeDaysLeg1'] - row['QALeadTimeDaysLeg2'] - row['TransitLeadTimeDaysLeg2']) if (row['QAStatusLeg3'] == 'XTOK') & (str(row['Flow'])[:4] == 'HALB') else
                0, axis=1)
merged_df['QALeadTimeDaysLeg3'] = merged_df.apply(lambda row: 0 if row['QALeadTimeDaysLeg3'] < 0 else row['QALeadTimeDaysLeg3'], axis=1)

merged_df['TotalLeadTimeDaysUnR'] = merged_df.apply(
    lambda row: row['Total Lead Time_Prepo3Site'] if (row['Flow'] == 'FERT') & (pd.isna(row['ProductionLocation'])) else
                row['Total Lead Time'] if row['Flow'] == 'FERT' else
                row['Total Lead Time_Prepo3'] if str(row['Flow'])[:4] == 'HALB' else
                0, axis=1)

merged_df['RemainingLeadTimeDaysUnR'] = merged_df.apply(
    lambda row: (row['TotalLeadTimeDaysUnR'] - row['QALeadTimeDaysLeg1'] - row['TransitLeadTimeDaysLeg1']
                - row['QALeadTimeDaysLeg2'] - row['TransitLeadTimeDaysLeg2'] - row['QALeadTimeDaysLeg3']
                - row['TransitLeadTimeDaysLeg3'] + row['ExportDocumentation'] + row['CustomTime']), axis=1)
merged_df['RemainingLeadTimeDaysUnR'] = merged_df.apply(lambda row: 0 if row['RemainingLeadTimeDaysUnR'] < 0 else row['RemainingLeadTimeDaysUnR'], axis=1)

# Condition for when XTOK is empty in the QA file
merged_df['QALeadTimeDaysLeg1'] = merged_df.apply(
    lambda row: row['RemainingLeadTimeDaysUnR'] + row['TransitLeadTimeDaysLeg1'] - row['ExportDocumentation'] - row['CustomTime']  if (row['QALeadTimeDaysLeg1'] == 0) & (row['QAStatusLeg1'] == 'XTOK') & (((row['Flow'] == 'FERT') & (row['Total Lead Time'] > 0) & (row['XTOK Time'] == 0)) or ((str(row['Flow'])[:4] == 'HALB') & (row['Total Lead Time_Whs1'] > 0) & (row['XTOK Time_Whs1'] == 0)))
    else row['QALeadTimeDaysLeg1'], axis=1)

merged_df['QAStatusLeg1'] = merged_df.apply(
    lambda row: 'UnRes' if (row['QALeadTimeDaysLeg1'] == row['RemainingLeadTimeDaysUnR'] + row['TransitLeadTimeDaysLeg1'] - row['ExportDocumentation'] - row['CustomTime']) & (row['QAStatusLeg1'] == 'XTOK') & (((row['Flow'] == 'FERT') & (row['Total Lead Time'] > 0) & (row['XTOK Time'] == 0)) or ((str(row['Flow'])[:4] == 'HALB') & (row['Total Lead Time_Whs1'] > 0) & (row['XTOK Time_Whs1'] == 0)))
    else row['QAStatusLeg1'], axis=1)

merged_df['QALeadTimeDaysLeg2'] = merged_df.apply(
    lambda row: row['RemainingLeadTimeDaysUnR'] + row['TransitLeadTimeDaysLeg2'] - row['ExportDocumentation'] - row['CustomTime'] if (row['QALeadTimeDaysLeg2'] == 0) & (row['QAStatusLeg2'] == 'XTOK') & (((row['Flow'] == 'FERT') & (row['Total Lead Time'] > 0) & (row['XTOK Time'] == 0)) or ((str(row['Flow'])[:4] == 'HALB') & (row['Total Lead Time_Whs2'] > 0) & (row['XTOK Time_Whs2'] == 0)))
    else row['QALeadTimeDaysLeg2'], axis=1)

merged_df['QAStatusLeg2'] = merged_df.apply(
    lambda row: 'UnRes' if (row['QALeadTimeDaysLeg2'] == row['RemainingLeadTimeDaysUnR'] + row['TransitLeadTimeDaysLeg2'] - row['ExportDocumentation'] - row['CustomTime']) & (row['QAStatusLeg2'] == 'XTOK') & (((row['Flow'] == 'FERT') & (row['Total Lead Time'] > 0) & (row['XTOK Time'] == 0)) or ((str(row['Flow'])[:4] == 'HALB') & (row['Total Lead Time_Whs2'] > 0) & (row['XTOK Time_Whs2'] == 0)))
    else row['QAStatusLeg2'], axis=1)

merged_df['QALeadTimeDaysLeg3'] = merged_df.apply(
    lambda row: row['RemainingLeadTimeDaysUnR'] + row['TransitLeadTimeDaysLeg3'] - row['ExportDocumentation'] - row['CustomTime'] if (row['QALeadTimeDaysLeg3'] == 0) & (row['QAStatusLeg3'] == 'XTOK') & (((row['Flow'] == 'FERT') & (row['Total Lead Time'] > 0) & (row['XTOK Time'] == 0)) or ((str(row['Flow'])[:4] == 'HALB') & (row['Total Lead Time_Prepo3'] > 0) & (row['XTOK Time_Prepo3'] == 0)))
    else row['QALeadTimeDaysLeg3'], axis=1)

merged_df['QAStatusLeg3'] = merged_df.apply(
    lambda row: 'UnRes' if (row['QALeadTimeDaysLeg3'] == row['RemainingLeadTimeDaysUnR'] + row['TransitLeadTimeDaysLeg3'] - row['ExportDocumentation'] - row['CustomTime']) & (row['QAStatusLeg3'] == 'XTOK') & (((row['Flow'] == 'FERT') & (row['Total Lead Time'] > 0) & (row['XTOK Time'] == 0)) or ((str(row['Flow'])[:4] == 'HALB') & (row['Total Lead Time_Prepo3'] > 0) & (row['XTOK Time_Prepo3'] == 0)))
    else row['QAStatusLeg3'], axis=1)

merged_df['RemainingLeadTimeDaysUnR'] = merged_df.apply(
    lambda row: row['ExportDocumentation'] + row['CustomTime'] if ((row['QALeadTimeDaysLeg1'] == row['RemainingLeadTimeDaysUnR'] + row['TransitLeadTimeDaysLeg1'] - row['ExportDocumentation'] - row['CustomTime']) & (row['QAStatusLeg1'] == 'UnRes') & (((row['Flow'] == 'FERT') & (row['Total Lead Time'] > 0) & (row['XTOK Time'] == 0)) or ((str(row['Flow'])[:4] == 'HALB') & (row['Total Lead Time_Whs1'] > 0) & (row['XTOK Time_Whs1'] == 0))))
                  or ((row['QALeadTimeDaysLeg2'] == row['RemainingLeadTimeDaysUnR'] + row['TransitLeadTimeDaysLeg2'] - row['ExportDocumentation'] - row['CustomTime']) & (row['QAStatusLeg2'] == 'UnRes') & (((row['Flow'] == 'FERT') & (row['Total Lead Time'] > 0) & (row['XTOK Time'] == 0)) or ((str(row['Flow'])[:4] == 'HALB') & (row['Total Lead Time_Whs2'] > 0) & (row['XTOK Time_Whs2'] == 0))))
                  or ((row['QALeadTimeDaysLeg3'] == row['RemainingLeadTimeDaysUnR'] + row['TransitLeadTimeDaysLeg3'] - row['ExportDocumentation'] - row['CustomTime']) & (row['QAStatusLeg3'] == 'UnRes') & (((row['Flow'] == 'FERT') & (row['Total Lead Time'] > 0) & (row['XTOK Time'] == 0)) or ((str(row['Flow'])[:4] == 'HALB') & (row['Total Lead Time_Prepo3'] > 0) & (row['XTOK Time_Prepo3'] == 0))))
    else row['RemainingLeadTimeDaysUnR'], axis=1)

merged_df['ProductionLocation'] = merged_df.apply(
    lambda row: pd.NA if pd.isna(row['ProductionLocation']) else
                str(row['ProductionLocation'])[:4] + '_SUP' if str(row['Flow'])[:4] == 'HALB' else
                str(row['ProductionLocation'])[:4], axis=1)

# Drop unecessary columns
merged_df = merged_df.drop(columns=['Fiscal year', 'SPECIE', 'ProcessingCountry', 'plant', 'DestinationCountry',
            'Stage', 'DTOK Time', 'XTOK Time', 'Total Lead Time', 'ExportDocumentation', 'CustomTime', 'Fiscal year_Whs1', 'Fiscal year_Whs2', 'Fiscal year_Prepo3',
            'SPECIE_Whs1', 'SPECIE_Whs2', 'SPECIE_Prepo3', 'ProcessingCountry_Whs1', 'ProcessingCountry_Whs2', 'ProcessingCountry_Prepo3', 'plant_Whs1', 'plant_Whs2', 'plant_Prepo3',
            'DestinationCountry_Whs1', 'DestinationCountry_Whs2', 'DestinationCountry_Prepo3', 'Stage_Whs1', 'Stage_Whs2', 'Stage_Prepo3', 'Total Lead Time_Whs1', 'Total Lead Time_Whs2',
            'Total Lead Time_Prepo3', 'DTOK Time_Whs1', 'DTOK Time_Whs2', 'DTOK Time_Prepo3', 'XTOK Time_Whs1', 'XTOK Time_Whs2', 'XTOK Time_Prepo3', 'PrepoDCLeg3Plant',
            'Fiscal year_Prepo3Site', 'SPECIE_Prepo3Site', 'ProcessingCountry_Prepo3Site', 'plant_Prepo3Site', 'DestinationCountry_Prepo3Site', 'Stage_Prepo3Site',
            'Total Lead Time_Prepo3Site', 'DTOK Time_Prepo3Site', 'XTOK Time_Prepo3Site'])

# Field conversions
merged_df['ProductionLocation'] = merged_df['ProductionLocation'].astype(pd.StringDtype())
merged_df['TotalLeadTimeDaysUnR'] = merged_df['TotalLeadTimeDaysUnR'].fillna(0).astype(int)
merged_df['RemainingLeadTimeDaysUnR'] = merged_df['RemainingLeadTimeDaysUnR'].fillna(0).astype(int)
merged_df['QALeadTimeDaysLeg1'] = merged_df['QALeadTimeDaysLeg1'].astype(int)
merged_df['QALeadTimeDaysLeg2'] = merged_df['QALeadTimeDaysLeg2'].astype(int)
merged_df['QALeadTimeDaysLeg3'] = merged_df['QALeadTimeDaysLeg3'].astype(int)
merged_df['FutureSeason'] = merged_df['FutureSeason'].astype(pd.StringDtype())
merged_df['OverflowWhsLeg1Site'] = merged_df['OverflowWhsLeg1Site'].astype(pd.StringDtype())
merged_df['OverflowWhsLeg2Site'] = merged_df['OverflowWhsLeg2Site'].astype(pd.StringDtype())
merged_df['PrepoDCLeg3Site'] = merged_df['PrepoDCLeg3Site'].astype(pd.StringDtype())


In [27]:
merged_df[merged_df['PrepoDCLeg3Site'] == '5901_FR']

,Specie,Flow,COD,ProductionCountry,ProductionLocation,QAStatusLeg1,QALeadTimeDaysLeg1,TransitLeadTimeDaysLeg1,OverflowWhsLeg1Country,OverflowWhsLeg1Site,...,PrepoDCLeg3Country,PrepoDCLeg3Site,TotalLeadTimeDaysUnR,CustomsLeadTimeDays,CertificationLeadTimeDays,FumigationLeadTimeDays,RemainingLeadTimeDaysUnR,LeadTimeDaysTotal,InSeason,FutureSeason
52,C,FERT,FR,FR,<NA>,,0,0,NaN,<NA>,...,FR,5901_FR,22,0,0,,22,0,Y,Y
